<a href="https://colab.research.google.com/github/hikkaaaa/machine-learning/blob/main/handling_missing_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1st task:
# Clean up the data
I got the idea from a 5 day missing values clean up challenge from kaggle

In [5]:
import numpy as np
import pandas as pd

#the dataset was taken from kaggle
#the data is about San Francisko permits
df = pd.read_csv("./drive/MyDrive/Colab Notebooks/Building_Permits.csv")

#for reproductibility:
np.random.seed(0)

/tmp/ipython-input-5-1166344860.py:6: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./drive/MyDrive/Colab Notebooks/Building_Permits.csv")


In [6]:
#take a look at some of the data:
df.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776


In [7]:
#how many missing values do we have in each column
missing_values_count = df.isnull().sum()
missing_values_count

#by analyzing the data, if we take 2 columns as examples, namely say: street number suffix and zipcode
#not all streets can have their number suffixes, so probably nan values in this field are simply bcus they dont exist
#however sipcode should be present for all streets, therfore the nan values in this column were probably forgotten to be recorded

,0
Permit Number,0
Permit Type,0
Permit Type Definition,0
Permit Creation Date,0
Block,0
Lot,0
Street Number,0
Street Number Suffix,196684
Street Name,0
Street Suffix,2768


In [8]:
#if you are in a hurry, drop the rows or columsn that contain missing values (however we will not do that, since analyzing the table is better)

#but just for education reasons, in case if you wanna drop the rows:
#df.dropna()
#in case if you want to drop all the columns that contain nan
#df.dropna(axis = 1)

In [9]:
#now lets try to get a subset that we gonna be working with and try to fill in some data to nan values
subset = df.loc[:, 'Street Number Suffix':'Zipcode'].head()
subset

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0
2,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN,...,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0


In [10]:
#replacing all the NA's with the value that comes after it in the same column
subset.fillna(method = "bfill", inplace = True)
subset

/tmp/ipython-input-10-3879349578.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset.fillna(method = "bfill", inplace = True)
/tmp/ipython-input-10-3879349578.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subset.fillna(method = "bfill", inplace = True)


,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,NaN,Ellis,St,0.0,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,NaN,3.0,constr type 3,1.0,constr type 1,NaN,3.0,Tenderloin,94102.0
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,NaN,3.0,constr type 3,1.0,constr type 1,NaN,3.0,Tenderloin,94102.0
2,NaN,Pacific,Av,0.0,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017,...,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0


In [11]:
#now lets fill the remaining NA's (the ones that didnt have any values to replace themselves with) with 0
subset.fillna(0, inplace = True)
subset

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,0.0,Ellis,St,0.0,0.0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0
1,0.0,Geary,St,0.0,0.0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0
2,0.0,Pacific,Av,0.0,0.0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017,...,2.0,0.0,1.0,constr type 1,1.0,constr type 1,0.0,3.0,Russian Hill,94109.0
3,0.0,Pacific,Av,0.0,0.0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,0.0,5.0,wood frame (5),5.0,wood frame (5),0.0,3.0,Nob Hill,94109.0
4,0.0,Market,St,0.0,0.0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,0.0,3.0,constr type 3,0.0,0,0.0,6.0,Tenderloin,94102.0


In [12]:
#thats pretty much about it for cleaning the data, the next parts will be just my amateur activities

# Converging data types

In [13]:
#inspecting what kind of a datatype each column has?:
df.dtypes
#for example: permit number represent an integer, however it is represents as an object
#now incase if youw ill be calculating with this data, u will need to convert it in any case, in case if not, leave it as object
#for now, we will convert it just too learn how to

,0
Permit Number,object
Permit Type,int64
Permit Type Definition,object
Permit Creation Date,object
Block,object
Lot,object
Street Number,int64
Street Number Suffix,object
Street Name,object
Street Suffix,object


In [14]:
#converting into integer/float - numeric
df['Permit Number'] = pd.to_numeric(df['Permit Number'], errors = 'coerce') #errors = 'coerce' just to be safe
df.dtypes

,0
Permit Number,float64
Permit Type,int64
Permit Type Definition,object
Permit Creation Date,object
Block,object
Lot,object
Street Number,int64
Street Number Suffix,object
Street Name,object
Street Suffix,object


In [15]:
#converting into string(object)
df['Street Number'] = df['Street Number'].astype(str)
df.dtypes

,0
Permit Number,float64
Permit Type,int64
Permit Type Definition,object
Permit Creation Date,object
Block,object
Lot,object
Street Number,object
Street Number Suffix,object
Street Name,object
Street Suffix,object


In [16]:
#or u can just do this super usefull method that will come up with the most suitable datatypes
df = df.convert_dtypes()
df.dtypes

,0
Permit Number,Int64
Permit Type,Int64
Permit Type Definition,string[python]
Permit Creation Date,string[python]
Block,string[python]
Lot,string[python]
Street Number,string[python]
Street Number Suffix,string[python]
Street Name,string[python]
Street Suffix,string[python]


# **Rename Columns**

In [17]:
df.rename(columns={
    'Permit Number': 'some kind of a new name',
    'Street Number': 'something else'
})

,some kind of a new name,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,something else,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,<NA>,Ellis,St,...,3,constr type 3,<NA>,<NA>,<NA>,3,Tenderloin,94102,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,<NA>,Geary,St,...,3,constr type 3,<NA>,<NA>,<NA>,3,Tenderloin,94102,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,<NA>,Pacific,Av,...,1,constr type 1,1,constr type 1,<NA>,3,Russian Hill,94109,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,<NA>,Pacific,Av,...,5,wood frame (5),5,wood frame (5),<NA>,3,Nob Hill,94109,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,<NA>,Market,St,...,3,constr type 3,<NA>,<NA>,<NA>,6,Tenderloin,94102,"(37.78315261897309, -122.40950883997789)",144548169992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,<NA>,8,otc alterations permit,12/05/2017,0113,017A,1228,<NA>,Montgomery,St,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1489337276729
198896,201712055595,8,otc alterations permit,12/05/2017,0271,014,580,<NA>,Bush,St,...,5,wood frame (5),5,wood frame (5),<NA>,<NA>,<NA>,<NA>,<NA>,1489462354993
198897,<NA>,8,otc alterations permit,12/06/2017,4318,019,1568,<NA>,Indiana,St,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1489539379952
198898,<NA>,8,otc alterations permit,12/06/2017,0298,029,795,<NA>,Sutter,St,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1489608233656


# **Check for weird indexes**

In [18]:
#check what kind of indexes does the DF have:
df.index

RangeIndex(start=0, stop=198900, step=1)

In [19]:
#in case if it is not 0, 1, 2, ... and not the one u want, u can reset it
df.reset_index(drop = True, inplace = True)
df.sample(2)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
60635,201409105988,3,additions alterations or repairs,09/10/2014,0479,027,1356,<NA>,Chestnut,St,...,5,wood frame (5),5,wood frame (5),Y,2,Marina,94123,"(37.802444119694016, -122.42570568145257)",1355098392583
110430,201601046309,8,otc alterations permit,01/04/2016,1276,001H,202,<NA>,Grattan,St,...,5,wood frame (5),5,wood frame (5),<NA>,5,Haight Ashbury,94117,"(37.7639230255061, -122.45133897567636)",1408159479509


# **Extracting some stuff**

In [20]:
#most frequent category in a  column
df['Permit Number'].value_counts().head(1)

,count
Permit Number,
201602179765,101


In [32]:
#the row with a max value in a numeric column
df.loc[df['Permit Number'].idxmax()]

,198885
Permit Number,201802232173
Permit Type,8
Permit Type Definition,otc alterations permit
Permit Creation Date,02/23/2018
Block,2150
Lot,001A
Street Number,2007
Street Number Suffix,<NA>
Street Name,30th
Street Suffix,Av


In [34]:
#mean of numeric by group
df.groupby('Permit Type').mean(numeric_only = True)

,Permit Number,Unit,Number of Existing Stories,Number of Proposed Stories,Estimated Cost,Revised Cost,Existing Units,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Record ID
Permit Type,,,,,,,,,,,,,,
1,201524711034.83667,0.0,<NA>,8.880117,36212794.580925,53744731.917476,<NA>,96.267974,2.031519,<NA>,1.571014,7.0,94113.26513,1227739573696.266357
2,201489118920.98526,0.4,1.0,3.162447,1937475.255711,2154835.198234,0.0,6.345898,2.001053,5.0,4.949314,7.415789,94120.778947,1292525615128.344238
3,201525988583.195587,37.684373,3.941568,4.256734,407119.308153,543680.018769,14.964355,18.248135,1.981925,4.32749,4.280521,5.936397,94115.95158,1182449339254.181641
4,201505883040.224762,56.806691,6.79171,<NA>,4738.676912,4880.859752,29.212831,<NA>,1.997233,2.632632,<NA>,5.171117,94113.479621,1145763782516.519043
5,201447152443.681305,0.0,<NA>,<NA>,687963.582418,1229300.738636,<NA>,<NA>,1.956044,<NA>,<NA>,5.989011,94117.989011,1121072663211.67041
6,201514484210.834991,0.045455,1.699664,<NA>,72884.001675,114849.620853,1.140741,<NA>,0.983333,4.310287,<NA>,6.954849,94114.428094,1179142076483.353271
7,201531224938.436401,88.943396,6.0,<NA>,3468.513583,1825.653636,0.0,<NA>,1.939335,3.0,<NA>,5.236686,94112.639053,1174129919434.878662
8,201520525921.61554,81.277792,5.878428,5.908476,47724.567013,41247.660609,15.578577,16.222069,1.178284,4.075936,4.070219,5.494391,94115.482409,1159745901487.351318
